In [1]:
import MagmaPandas.volatile_solubility.models.IaconoMarziano as im
import MagmaPandas.volatile_solubility.models.Allison2022 as a
import MagmaPandas.volatile_solubility.models.shiskina as sh
sh_config = sh.shiskina_configuration
sh_config.print()
# print("\n")
# a_config = a.Allison_configuration
# a_config.print()

##### Shiskina volatile solubility #####
########################################
Settings________________________________
Fugacity model.....................ideal

Calibration range______________________
Temperature..................1423-1523°K
Pressure......................0.2-5 kbar




In [2]:
# import pandas as pd
import MagmaPandas as mp
import MagmaPandas.parse_io.readers as r
import pandas as pd
import elements as e
import numpy as np
import matplotlib.pyplot as plt
import VESIcal as v
config = mp.configuration
print(config)


################ MagmaPandas ################
#############################################
General settings_____________________________
fO2 buffer................................QFM
Δ buffer....................................1
Melt Fe3+/Fe2+........................borisov
Kd Fe-Mg ol-melt.......................toplis
Melt thermometer...............putirka2008_15
Volatile solubility model......IaconoMarziano
Volatile species........................mixed


/Users/thomas/miniforge3/lib/python3.9/site-packages/VESIcal/calculate_classes.py:7: UserWarning: 

  from VESIcal.models import magmasat


In [3]:

melt = mp.read_melt('./melt.csv', index_col=['name'], total_col='total')

In [5]:
config.volatile_solubility = "shiskina"
melt.volatile_saturation_pressure(1400)

|██████████████████████████████| 29/29 [100%] in 7.6s (3.79/s)                                                          


name
PI032-01-01    3949.366305
PI032-01-04    3057.190217
PI032-02-01    4341.186656
PI032-02-02    3952.830863
PI032-02-03    4150.502879
PI032-03-01    3496.658999
PI032-04-01    5501.409275
PI032-04-02    5970.260092
PI032-04-03     773.562849
PI032-05-01    3804.514753
PI041-02-01    4589.353936
PI041-02-02    2716.185743
PI041-03-01    4407.296932
PI041-03-03    4363.360713
PI041-04-02    5534.463719
PI041-04-03    5583.160472
PI041-04-04    5246.134624
PI041-04-05    5632.732573
PI041-04-06    5412.904744
PI041-04-07    5919.402605
PI041-05-02    2875.194412
PI041-05-03    2945.745299
PI041-05-04    2253.836206
PI041-05-06    3780.551695
PI041-07-01    4808.849925
PI041-07-02    2988.226375
PI041-07-06    3821.417536
PI052-01-02    1975.676346
PI052-02-01    2305.232334
dtype: float64

In [ ]:
sample = "PI032-04-01"
FeO_initial = 15
Fo_host = 0.6
Kd = 0.29
P_bar = 1e3
T_K = 1500
inclusion = melt.loc[sample,:].copy()
# inclusion["H2O"] = 5


In [ ]:
import MagmaPandas.volatile_solubility as vs
vs.calculate_saturation(inclusion, 1400)

In [ ]:
s = v.Sample(inclusion[inclusion.index.intersection(v.oxides)])
s.get_composition()

In [ ]:
v.get_model_names()

In [ ]:
print(f"{'Sample': <15}{'VESIcal':<10}{'Me':<10}{'delta': <7}{'X-H2O': >}")
for idx in range(20):

    inclusion = melt.iloc[idx,:]
    s = v.Sample(inclusion[inclusion.index.intersection(v.oxides)])
    model = "IaconoMarziano"
    VC = v.calculate_saturation_pressure(s, temperature=T_K-273, model=model).result
    IM, xIM = im.calculate_saturation(inclusion, T_K, output="both")
    # A, xA = a.calculate_saturation(inclusion, T_K, output="both")
    # SH, x = sh.mixed.calculate_saturation(inclusion, output="both")
    # string = f"IM: {IM:>5.0f}, A: {A:>5.0f}"    
    print(f"{inclusion.name:<15}{VC:<10.0f}{IM:<10.0f}{(VC - IM): <7.0f}{xIM: >.2f}")
    # print(f"VESIcal: {VC:<10.0f} {string:>25}, delta: {(VC - IM): >3.0f} bar, Xfl-IM: {xim: >4.2f}, Xfl-A: {x: .2f}")

In [ ]:
model = "IaconoMarzianoWater"

p = np.arange(1,  1e4, 5e1)
fig, ax = plt.subplots(figsize=(10, 10))

plt.plot(p, [im.h2o.calculate_solubility(inclusion, P_bar=i, T_K=T_K) for i in p], label="Me", linewidth=4, alpha=0.8, linestyle="--")
plt.plot(p, [v.calculate_dissolved_volatiles(s, temperature=T_K-273.15, pressure=i, model=model).result for i in p], label="VESIcal")

plt.legend()
plt.show()

In [ ]:
melt.iloc[0, :]